In [ ]:
TrainDataSet = pd.read_csv('./datasets/train_kaggle.csv')

TrainDataSet.info()

# Check for missing values
TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']] = TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']].fillna(0)

TrainDataSet.dropna(inplace=True)
TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']] = TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']].apply(pd.to_numeric, errors='coerce')

# Convert the columns to float
TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']] = TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']].astype(float)

TrainDataSet.replace({'luxury_sales': {np.nan: 0}, 'fresh_sales': {np.nan: 0}, 'dry_sales': {np.nan: 0}}, inplace=True)

# Convert the columns to float
TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']] = TrainDataSet[['luxury_sales', 'fresh_sales', 'dry_sales']].astype(float)

# One Hot Encoding
TrainDataSet = pd.get_dummies(TrainDataSet, columns=['outlet_city'])

# # Copy the target column
Target = TrainDataSet['cluster_catgeory']

# Drop the original target column from TrainDataSet
TrainDataSet.drop(['cluster_catgeory'], axis=1, inplace=True)

# Concatenate the target column to the end of TrainDataSet
TrainDataSet['cluster_catgeory'] = Target

# Convert strings to numeric (excluding unusual entries)
y_train_numeric = pd.to_numeric(TrainDataSet['cluster_catgeory'], errors='coerce')

# Remove NaNs resulting from conversion
y_train_numeric = y_train_numeric[~np.isnan(y_train_numeric)]

# Ensure consistent data type (optional)
y_train_numeric = y_train_numeric.astype(int)

# Replace the 'cluster_catgeory' column in TrainDataSet with the cleaned numeric values
TrainDataSet['cluster_catgeory'] = y_train_numeric

# Define the list of outlier values
outliers = [95, 98, 99, 100, 89]

# Filter out rows with outlier values from 'cluster_catgeory'
TrainDataSet = TrainDataSet[~TrainDataSet['cluster_catgeory'].isin(outliers)]

# Drop rows with NaN values in the 'cluster_catgeory' column
TrainDataSet.dropna(subset=['cluster_catgeory'], inplace=True)

# Get unique values
unique_classes = TrainDataSet['cluster_catgeory'].unique()


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define preprocessing steps
numeric_features = [...]  # List of numeric feature column names
categorical_features = [...]  # List of categorical feature column names

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define model
model = LogisticRegression(solver='lbfgs', max_iter=250)

# Create pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train pipeline
pipeline.fit(X_train, y_train)

# Predictions
y_pred = pipeline.predict(X_test)

# Evaluate pipeline
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
